In [ ]:
import pandas as pd
test_data=pd.read_csv("/content/taskC_test_edited.csv")
test_data.head()

,index,tweet,truth_label
0,50008,सबै मिली सत्रे लाई हराऔँ🙏🙏\n#NoNotAgain \n@cmp...,2
1,50013,केही बर्षमा कांग्रेस भित्र माओबादी मात्रै बाकी...,1
2,50017,"कस्ता पार्टी सभापति हुन्, हँसियाहथौडामा भाेट न...",0
3,50018,@prakashgiriji नेपाली जनता श्रीलंकाको जनता बन्...,2
4,50036,दलबदलुओं को टिकट देने पर मणिपुर बीजेपी में बगा...,0


In [ ]:
test_tweets=test_data['tweet'].tolist()

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load the models and tokenizers
distilbert_model = DistilBertForSequenceClassification.from_pretrained('/content/fine_tuned_distillbert_taskc', num_labels=3,from_tf=True)
bert_model = BertForSequenceClassification.from_pretrained('/content/fine_tuned_model_gb', num_labels=3,from_tf=True)

distilbert_tokenizer = DistilBertTokenizer.from_pretrained('/content/fine_tuned_distillbert_taskc')
bert_tokenizer = BertTokenizer.from_pretrained('/content/fine_tuned_model_gb')

# Sample input data (replace this with your actual test data)
input_texts = test_tweets
# Define label mapping
label_mapping = {
    0: "individual",  # Replace with your actual label
    1: "organization",   # Replace with your actual label
    2: "community"   # Replace with your actual label
}

# Tokenize the inputs
distilbert_inputs = distilbert_tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True)
bert_inputs = bert_tokenizer(input_texts, return_tensors='pt', padding=True, truncation=True)

# Generate predictions
with torch.no_grad():
    distilbert_outputs = distilbert_model(**distilbert_inputs)
    bert_outputs = bert_model(**bert_inputs)

# Get predicted probabilities
distilbert_probs = torch.nn.functional.softmax(distilbert_outputs.logits, dim=-1)
bert_probs = torch.nn.functional.softmax(bert_outputs.logits, dim=-1)

# Combine predictions using equal weights (simple average)
combined_probs = (distilbert_probs + bert_probs) / 2

# Get final predictions
final_predictions = torch.argmax(combined_probs, dim=1)

# Convert predictions to labels
labels = final_predictions.tolist()  # Convert tensor to list for easier handling

# Print the output labels with mapping
for text, label_index in zip(input_texts, labels):
    label = label_mapping[label_index]  # Map index to label
    print(f"Input: {text} => Predicted Label: {label} (Index: {label_index})")

All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.
All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


Input: सबै मिली सत्रे लाई हराऔँ🙏🙏
#NoNotAgain 
@cmprachanda => Predicted Label: community (Index: 2)
Input: केही बर्षमा कांग्रेस भित्र माओबादी मात्रै बाकी रलान जस्तो छ,

राजीनामा राजीनामा,  निष्कासन  गर्न थाले कांग्रेसि 😂 => Predicted Label: community (Index: 2)
Input: कस्ता पार्टी सभापति हुन्, हँसियाहथौडामा भाेट नहाले कारबाही रे ।
#मारेके => Predicted Label: community (Index: 2)
Input: @prakashgiriji नेपाली जनता श्रीलंकाको जनता बन्न समय लाग्छ। नेपाली जनताले दुख भोग्न पुगेको छैन। => Predicted Label: community (Index: 2)
Input: दलबदलुओं को टिकट देने पर मणिपुर बीजेपी में बगावत, पीएम मोदी के पुतले फूंके गए -

#Manipur #ManipurElections2022 https://t.co/qMQZpK7zIB via @SatyaHindi => Predicted Label: community (Index: 2)
Input: एग्जिट पोल पर लगाना चाहिए बैन, AAP ने पैसे दे कर करवाया सर्वे: अकाली दल मुखिया सुखबीर सिंह बादल

#PunjabElections2022 #ExitPolls #AAP

https://t.co/kmuJQqPb7c => Predicted Label: community (Index: 2)
Input: @madhuBTM276 पुर्व सचीवको एमाले संग कुनै न कुनै प्रतिशोध छ ।

In [ ]:
# prompt: classification report for the above predictions

from sklearn.metrics import classification_report

# Assuming 'true_labels' is a list or numpy array of the true labels for your test data
# Replace this with your actual true labels
true_labels = test_data['truth_label'].tolist() # Assuming your true labels are in a column named 'label'


print(classification_report(true_labels, labels))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       226
           1       0.00      0.00      0.00       182
           2       0.13      1.00      0.23        61

    accuracy                           0.13       469
   macro avg       0.04      0.33      0.08       469
weighted avg       0.02      0.13      0.03       469



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
